<a href="https://colab.research.google.com/github/aiatoz/NLP/blob/main/WebQueryEngine/base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This is an open source project to ground an LLM on Web Data for querying

Due to limited compute I've restricted the context length to 2048. Feel free to increase it if you've enough computing power. The default context length of 2048 will work just fine with a T4 GPU

For more details reagrding the libraries used, please refer

Url lib : https://pypi.org/project/urllib3/

BeautifulSoup : https://pypi.org/project/beautifulsoup4/

ctransformers : https://pypi.org/project/ctransformers/

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Oct 28 14:18:34 2023

@author: Krishna
"""


#Installing dependencies

!pip install urllib3
!pip install ctransformers[cuda]

Scraping and filtering out data

In [1]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib3


def filterTag(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def htmlToText(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.find_all(string=True)
    visible_texts = filter(filterTag, texts)
    return u" ".join(t.strip() for t in visible_texts)

req = urllib3.PoolManager()
response = req.request('GET', "https://atozofai.withgoogle.com/intl/en-US/artificial-intelligence/")
htmlContent = response.data

scrapedText = htmlToText(htmlContent)
print(scrapedText)


           The A-Z of AI  This site requires JavaScript - you can usually enable this in your
          browser settings.      The A-Z of AI  Artificial intelligence  At its heart, AI is computer programming that learns and adapts.  Artificial intelligence is the name given to any computer system
          taught to mimic intelligent human behaviors.  It refers to anything from programs taught how to play board games and
          translate languages to complex systems that can have human-sounding
          conversations, analyze the stock market or help to diagnose health
          conditions.  AI is powering practical tools that exist all around us.  These technologies already show up in everyday life — helping to
          organize photos on our smartphones or plan a commute to work. They
          take tasks that have historically required considerable human
          intellect and work through them using computer programming.  The last decade has seen major progress, sparked by in

#Loading and using the LLM

The LLM used here is Mistral 7b which is a light LLM that can even run on a consumer end GPU

In [3]:
from ctransformers import AutoModelForCausalLM
llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GGUF", model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf", model_type = "mistral", gpu_layers = 50, context_length=2048)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
query = input("Enter the question :")

Content = """<s>[INST]
The data enclosed in tilde symbol is a web scraped output. Avoid the unnecessary parts such as 'Jump to content', 'Main menu', 'Navigation' etc,
that usually occures in websites.

Based on the data answer the queries.

Web Data : ~
{}
~

Query : {}

[/INST]""".format(scrapedText, query)

#To enable chat, you can just append the messages, the context will act like a memory, and clearing it will automatically wipe the memory out
#print(llm(Content))

print(llm(Content))

Enter the question :Summarize the page
 The page discusses the concept of artificial intelligence (AI) and its applications in various fields such as gaming, language translation, stock market analysis, health diagnosis, smartphones, and commute planning. AI technologies have been improving due to increased computer speeds and machine learning techniques, and are becoming increasingly integrated into daily life. The future of these technologies will depend on how people use them, but their potential to transform human challenges is clear.
